In [ ]:
import pandas as pd
import json
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
1.2335434230000146
GPU (s):
0.08744279100000085
GPU speedup over CPU: 14x


In [ ]:
# import pandas as pd
# import tensorflow as tf
# from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
# import seaborn as sns
# from tqdm.notebook import tqdm

# # Eager Executionモードを無効にする
# tf.compat.v1.disable_eager_execution()

# # GPUデバイスのメモリ成長を設定
# physical_devices = tf.config.list_physical_devices('GPU')
# if len(physical_devices) > 0:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)

# # TensorFlowセッションの初期化
# tf.compat.v1.Session()

# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 300)
# pd.set_option('display.max_colwidth', 300)
# sns.set_style("whitegrid")
# colors_nude = ['#e0798c','#65365a','#da8886','#cfc4c4','#dfd7ca']
# %matplotlib inline
# tqdm.pandas()

#以下は使用メモリの確認。Google が Colabの使い方で教えているコード
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun May 12 12:34:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0              29W /  70W |   1215MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# version 確認　（2019/04/10 時点で cuda プレインストールされている）
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/2_DPTUPCDR')
!ls -la

total 120
-rw------- 1 root root   903 May  3 02:36 config.json
-rw------- 1 root root   940 Nov  8  2023 config.json.org
drwx------ 2 root root  4096 May  3 02:07 data
-rw------- 1 root root 74781 May 12 12:34 DPTUPCDR.ipynb
-rw------- 1 root root  1795 Nov  8  2023 entry.py
-rw------- 1 root root  4975 May 12 06:26 models_gMLP_only.py
-rw------- 1 root root  6176 May 11 17:09 models.py
-rw------- 1 root root  3838 May 11 04:16 models.py.org
-rw------- 1 root root  3711 Nov  8  2023 preprocessing.py
drwx------ 2 root root  4096 May  3 02:11 __pycache__
-rw------- 1 root root  3549 Nov  8  2023 README.md
-rw------- 1 root root 10904 Nov  8  2023 run.py


In [9]:
!python entry.py --process_data_mid 1 --process_data_ready 1
#最初からやる場合は、./data フォルダの ready フォルダを削除する
# 31分で完了

2024-05-12 13:07:23.706283: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 13:07:23.706334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 13:07:23.707532: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-12 13:07:24.686162: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Parsing Books Mid...
8898041it [02:33, 58093.33it/s]
Books Mid Done.
Parsing CDs_and_Vinyl Mid...
1097592it [00:20, 53014.84it/s]
CDs_and_Vinyl Mid Done.
Parsing Movies_and_TV Mid...
1697533it [00:31, 54043.47it/s]
Movies_and_TV Mid Done.
So

In [ ]:
#!ls ./data/raw/reviews_Books_5.json.gz

./data/raw/reviews_Books_5.json.gz


In [ ]:
#!pwd

/content/drive/MyDrive/Colab Notebooks/1_PTUPCDR


In [ ]:
# !python entry.py
# # 13分で終わる, 2回目 21分かかった（プリプロセスなしでできるか不明）

2024-05-11 03:51:32.959760: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 03:51:32.959817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 03:51:32.961664: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-11 03:51:34.008031: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
task:1, ratio:[0.8, 0.2], epoch:15, lr:0.01, gpu:0, seed:2020
========Reading data========
src 6631 iter / batchsize = 256 
tgt 3951 iter / batchsize = 256 
meta 2983 iter / batchsize = 128 
map 226 iter / batchsize = 64 
aug 10582 iter / ba

In [ ]:
# !python entry.py
# # gMLPに置き換えただけ

2024-05-11 10:25:33.618337: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 10:25:33.618394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 10:25:33.619572: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-11 10:25:34.671833: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
task:1, ratio:[0.8, 0.2], epoch:15, lr:0.01, gpu:0, seed:2020
========Reading data========
src 6631 iter / batchsize = 256 
tgt 3951 iter / batchsize = 256 
meta 2983 iter / batchsize = 128 
map 226 iter / batchsize = 64 
aug 10582 iter / ba

In [ ]:
# !python entry.py

2024-05-11 14:07:31.542900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 14:07:31.542957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 14:07:31.544819: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-11 14:07:32.604598: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
task:1, ratio:[0.8, 0.2], epoch:15, lr:0.01, gpu:0, seed:2020
========Reading data========
src 6631 iter / batchsize = 256 
tgt 3951 iter / batchsize = 256 
meta 2983 iter / batchsize = 128 
map 226 iter / batchsize = 64 
aug 10582 iter / ba

In [ ]:
!python entry.py --epoch 1

2024-05-12 12:36:19.854140: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 12:36:19.854190: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 12:36:19.855481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-12 12:36:20.873941: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
task:1, ratio:[0.8, 0.2], epoch:1, lr:0.01, gpu:0, seed:2020
========Reading data========
src 6631 iter / batchsize = 256 
tgt 3951 iter / batchsize = 256 
meta 2983 iter / batchsize = 128 
map 226 iter / batchsize = 64 
aug 10582 iter / bat